In [106]:
a = [1,1,2,3,4,5,6,6,6,6,2]
a.count(8)

0

In [122]:
class groupCards:
  def __init__(self):
    self.card_numbers = {
        '2':0,'3':1,'4':2,'5':3,'6':4,'7':5,'8':6,'9':7,'10':8,
        'J':9,'Q':10,'K':11,'A':12
    }
    self.card_numbers_invert = {v:k for k, v in self.card_numbers.items()}

    self.straight_sets = self._create_straight_sets()

  def __call__(self, players_cards, floor_cards):
    self.players_cards = players_cards
    self.floor_cards = floor_cards
    self.all_floor_cards = sum(players_cards.values(), [])
    self.max()

  def _create_straight_sets(self):
    nums = list(self.card_numbers.values())
    st = []
    for i in range(len(nums)-5,-1,-1):  # Sorted from maximum straight
      s = []
      for j in range(i,i+5):
        s.append(nums[j])
      st.append(set(s))
    st.append({12,0,1,2,3})  # Ace,2,3,4,5
    return st

  def set_number_to_cards_value(self, c):
    return self.card_numbers[c[1:]]

  def max(self):
    cards = [card[1:] for card in self.all_floor_cards]
    maximum = max([self.card_numbers[card] for card in cards])
    self.players_max = [card for card in self.all_floor_cards if self.card_numbers[card[1:]] == maximum]

  def high_cards(self, players_cards):
    winners = []
    for player, cards in players_cards.items():
      for card in cards:
        if card in self.players_max:
          winners.append(player)
    return winners

  def check_high_card(self):
    winners = self.high_cards(self.players_cards)
    for _ in range(4):
      new_cards = {player:self.players_cards[player] for player in winners}
      winners = self.high_cards(new_cards)
    return winners[0]

  def one_pair_for_a_player(self, player_card):
    player_card_with_floor_cards = player_card+self.floor_cards
    numbers = [c[1:] for c in player_card_with_floor_cards]
    for i, n in enumerate(numbers[:-1]):
      if n in numbers[i+1:]:
        return True, n
    return False, None

  def check_one_pair(self):
    pair_cards_ = {'player1':None, 'player2':None, 'player3':None, 'player4':None}
    for p, cards in self.players_cards.items():
      ispair, pair_card = self.one_pair_for_a_player(cards)
      if ispair:
        pair_cards_[p] = pair_card
    pair_cards = {}
    for p, c in pair_cards_.items():
      if c is not None:
        pair_cards[p] = c
    del pair_cards_
    if pair_cards:
      return max(pair_cards, key=lambda x: self.card_numbers[pair_cards[x]])
    else:
      return None

  def two_pairs_for_a_player(self, player_card):
    player_card_with_floor_cards = player_card+self.floor_cards
    numbers = [c[1:] for c in player_card_with_floor_cards]
    count = {}
    for num in numbers:
      if num in count:
        count[num] += 1
      else:
        count[num] = 1
    pairs = [num for num, cnt in count.items() if cnt >= 2]
    if len(pairs)==2:
      return True, pairs
    else:
      return False, None

  def check_two_pairs(self):
    pair_cards_ = {'player1':None, 'player2':None, 'player3':None, 'player4':None}
    for p, cards in self.players_cards.items():
      ispair, pair_card = self.two_pairs_for_a_player(cards)
      if ispair:
        pair_cards_[p] = pair_card
    pair_cards = {}
    for p, c in pair_cards_.items():
      if c is not None:
        pair_cards[p] = c
    del pair_cards_
    if pair_cards:
      maximum = max(sum(pair_cards.values(), []))
      for p, c in pair_cards.items():
        if maximum in c:
          return p
    else:
      return None

  def three_of_a_kind_for_a_player(self, player_card):
    player_card_with_floor_cards = player_card+self.floor_cards
    numbers = [c[1:] for c in player_card_with_floor_cards]
    count = {}
    for num in numbers:
      if num in count:
        count[num] += 1
      else:
        count[num] = 1
    pairs = [num for num, cnt in count.items() if cnt >= 3]
    if len(pairs):
      return True, pairs[0]
    else:
      return False, None

  def check_three_of_a_kind(self):
    pair_cards_ = {'player1':None, 'player2':None, 'player3':None, 'player4':None}
    for p, cards in self.players_cards.items():
      ispair, pair_card = self.three_of_a_kind_for_a_player(cards)
      if ispair:
        pair_cards_[p] = pair_card
    pair_cards = {}
    for p, c in pair_cards_.items():
      if c is not None:
        pair_cards[p] = c
    del pair_cards_
    if pair_cards:
      return max(pair_cards, key=lambda x: self.card_numbers[pair_cards[x]])
    else:
      return None

  def straight_for_a_player(self, player_card):
    player_card_with_floor_cards = player_card+self.floor_cards
    numbers = sorted([self.card_numbers[c[1:]] for c in player_card_with_floor_cards])

    for st in self.straight_sets:
      if st.issubset(numbers):
        if st=={12,0,1,2,3}:  # Ace,2,3,4,5
          return True, 3
        return True, max(st)
    return False, None

  def check_straight(self):
    straight_cards_ = {'player1':None, 'player2':None, 'player3':None, 'player4':None}
    for p, cards in self.players_cards.items():
      ispair, straight_card = self.straight_for_a_player(cards)
      if ispair:
        straight_cards_[p] = straight_card
    straight_cards = {}
    for p, c in straight_cards_.items():
      if c is not None:
        straight_cards[p] = c
    del straight_cards_
    if straight_cards:
      return max(straight_cards, key=lambda x: straight_cards[x])
    else:
      return None

  def flush_for_a_player(self, player_card):
    player_card_with_floor_cards = player_card+self.floor_cards
    groups = [c[0] for c in player_card_with_floor_cards]
    print(groups)
    for g in ["C", "D", "S", "H"]:
      if groups.count(g)>=5:
        maximum = max([self.card_numbers[c[1:]] for c in player_card_with_floor_cards if c.startswith(g)])
        return True, maximum

    return False, None

In [123]:
group_cards = groupCards()
cards = {
    'player1' : ["C2", 'CA'],
    'player2' : ["C5", 'C5'],
    'player3' : ["C7", 'C8'],
    'player4' : ["C2", 'C6'],
}
group_cards(cards, ['C4', 'C5', 'C3'])

In [125]:
# group_cards.flush_for_a_player(cards['player1'])
# group_cards.check_straight()

['C', 'C', 'C', 'C', 'C']


(True, 12)

In [98]:
group_cards.straight_sets

[{8, 9, 10, 11, 12},
 {7, 8, 9, 10, 11},
 {6, 7, 8, 9, 10},
 {5, 6, 7, 8, 9},
 {4, 5, 6, 7, 8},
 {3, 4, 5, 6, 7},
 {2, 3, 4, 5, 6},
 {1, 2, 3, 4, 5},
 {0, 1, 2, 3, 4},
 {0, 1, 2, 3, 12}]

In [ ]:
card_numbers = {
        '2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,'10':10,
        'J':11,'Q':12,'K':13,'A':14}

c = {
    'player1' : [4, 5],
    'player2' : [6, 7],
    'player3' : [8, 2],
    'player4' : [1, 1],
}
max(c, key=c.get)

'player3'

In [ ]:
max([1,2],[3,4])

[3, 4]

In [ ]:
# high card
def high_card(cards):

  all_floor_cards = sum(cards.values(), [])

  winner = [k for k,v in cards.items() if max_card in v]
  return winner
high_card(cards)

ValueError: ['C5', 'C9', 'CK', 'CA', 'CK', 'CA', 'CK', 'CA', 'CK', 'CA'] is not in list

['C2', 'C5', 'C9', 'CK', 'CA', 'CK', 'CA', 'CK', 'CA', 'CK', 'CA']

In [ ]:
sum([[1,2,3],[4,5]],[])

[1, 2, 3, 4, 5]